
# `Skore` - an abstraction to ease data science project

In [ ]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "true"


Let's open a skore project in which we will be able to store artifacts from our
experiments.

In [ ]:
import skore

my_project = skore.Project("../data/my_project", if_exists=True)

In [ ]:
from skrub.datasets import fetch_employee_salaries

datasets = fetch_employee_salaries()
df, y = datasets.X, datasets.y

In [ ]:
from skrub import TableReport

table_report = TableReport(datasets.employee_salaries)
table_report


Let's model our problem.

In [ ]:
from skrub import TableVectorizer, TextEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline

model = make_pipeline(
    TableVectorizer(high_cardinality=TextEncoder()),
    RandomForestRegressor(n_estimators=20, max_leaf_nodes=40),
)
model


`skore` provides a couple of tools to ease the evaluation of model:

In [ ]:
from skore import CrossValidationReport

report = CrossValidationReport(estimator=model, X=df, y=y, cv_splitter=5, n_jobs=-1)

In [ ]:
report.help()

In [ ]:
import time

start = time.time()
score = report.metrics.r2()
end = time.time()
print(f"Time taken: {end - start:.2f} seconds")

In [ ]:
score

In [ ]:
start = time.time()
score = report.metrics.r2()
end = time.time()
print(f"Time taken: {end - start:.2f} seconds")

In [ ]:
score

In [ ]:
import time

start = time.time()
score = report.metrics.rmse()
end = time.time()
print(f"Time taken: {end - start:.2f} seconds")

In [ ]:
score

In [ ]:
report.cache_predictions(n_jobs=-1)

In [ ]:
my_project.put("Random Forest model report", report)

In [ ]:
report = my_project.get("Random Forest model report")
report.help()

In [ ]:
report.metrics.report_metrics(aggregate=["mean", "std"], indicator_favorability=True)

In [ ]:
display = report.metrics.prediction_error()
display.plot(kind="actual_vs_predicted")

In [ ]:
report.estimator_reports_

In [ ]:
estimator_report = report.estimator_reports_[0]
estimator_report.help()

In [ ]:
estimator_report.metrics.prediction_error().plot(kind="actual_vs_predicted")

In [ ]:
import numpy as np
import skrub
from sklearn.linear_model import RidgeCV

model = skrub.tabular_learner(RidgeCV(np.logspace(-3, 3, 10)))
model

In [ ]:
report = CrossValidationReport(estimator=model, X=df, y=y, cv_splitter=5, n_jobs=-1)
my_project.put("RidgeCV model report", report)

In [ ]:
from skore import ComparisonReport

report = ComparisonReport(
    reports={
        "Random Forest": my_project.get("Random Forest model report"),
        "RidgeCV": my_project.get("RidgeCV model report"),
    },
)

In [ ]:
report.metrics.report_metrics(indicator_favorability=True)


## Conclusions

**Vision**
- Develop tooling to create data science artifacts
- Help at following good practices for the problem at hand
- Help at the collaboration to carry on data science project

**Wrap-up**
- Provide tools to evaluate predictive models
- Make some internal magic to reduce user friction
- Allow for persistence of artifacts

**Roadmap**
- Cover multiple aspects of the data science life cycles: data, model, etc.
- Help at creating artifacts dedicated to the problem at hand and the model
- Reduce the complexity related to code